<a href="https://colab.research.google.com/github/antbartash/australian_rain/blob/main/MicrosoftNNI/BOGP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Microsoft NNI on Google Colab

In [1]:
! pip install nni # install nni
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip # download ngrok and unzip it
! unzip ngrok-stable-linux-amd64.zip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 9.1 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
  Attempting uninstall: filelock
    Found existing installation: filelock 3.13.1
    Uninstalling filelock-3.13.1:
      Successfully uninstalled filelock-3.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.
--2023-12-23 14:0

In [2]:
! ./ngrok authtoken 2ZqSaEGuFWdJVe4YADpDuGtZ1R9_2L3tHZaAHSSavE6N96jo4 # YOUR_AUTH_TOKEN

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [3]:
# clone the github repo to get the model.py file
! git clone https://github.com/antbartash/australian_rain.git

Cloning into 'australian_rain'...
remote: Enumerating objects: 161, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 161 (delta 27), reused 0 (delta 0), pack-reused 101
Receiving objects: 100% (161/161), 8.22 MiB | 12.28 MiB/s, done.
Resolving deltas: 100% (72/72), done.


In [4]:
from nni.experiment import Experiment

In [5]:
hyperparameter_space = {
    'n_estimators': {'_type': 'randint', '_value': [100, 500]},
    'learning_rate': {'_type': 'loguniform', '_value': [1e-6, 0.5]},
    'depth': {'_type': 'randint', '_value': [1, 6]},
    'l2_leaf_reg': {'_type': 'uniform', '_value': [0.0, 100.0]},
    # 'random_strength': {'_type': 'uniform', '_value': [0.0, 100.0]}, # CPU only
    'bagging_temperature': {'_type': 'uniform', '_value': [0.0, 100.0]},
    # 'grow_policy': {'_type': 'choice', '_value': ['SymmetricTree', 'Depthwise']}, # GPTuner support only numerical values
    'scale_pos_weight': {'_type': 'uniform', '_value': [1, 4]}
}

In [6]:
experiment = Experiment('local')
experiment.config.experiment_name = 'catboost_nni_bogp'
experiment.config.tuner.name = 'GPTuner'
experiment.config.tuner.class_args = {
    'optimize_mode': 'maximize',
    'utility': 'ei', # aquisition function
    'xi': 0.0,
    'cold_start_num': 5
}
experiment.config.trial_command = 'python3 /content/australian_rain/MicrosoftNNI/model_GPU.py'
experiment.config.trial_code_directory = '.'
experiment.config.search_space = hyperparameter_space
experiment.config.max_trial_number = 10
experiment.config.trial_concurrency = 1
experiment.config.max_experiment_duration = '1h'

In [7]:
get_ipython().system_raw('./ngrok http 5000 &')

In [8]:
! curl -s http://localhost:4040/api/tunnels # don't change the port number 4040

{"tunnels":[{"name":"command_line (http)","uri":"/api/tunnels/command_line%20%28http%29","public_url":"http://cf3f-104-154-111-8.ngrok-free.app","proto":"http","config":{"addr":"http://localhost:5000","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}},{"name":"command_line","uri":"/api/tunnels/command_line","public_url":"https://cf3f-104-154-111-8.ngrok-free.app","proto":"https","config":{"addr":"http://localhost:5000","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}}],"uri":"/api/tunnels"}


In [9]:
experiment.run(5000, wait_completion=True, debug=False)

[2023-12-23 14:08:49] Creating experiment, Experiment ID: xl6s8fct


INFO:nni.experiment:Creating experiment, Experiment ID: ${CYAN}xl6s8fct


[2023-12-23 14:08:49] Starting web server...


INFO:nni.experiment:Starting web server...


[2023-12-23 14:08:51] Setting up...


...

INFO:nni.experiment:Saving experiment checkpoint...


[2023-12-23 14:12:39] Stopping NNI manager, if any...


INFO:nni.experiment:Stopping NNI manager, if any...


[2023-12-23 14:12:41] Experiment stopped.


INFO:nni.experiment:Experiment stopped.


In [ ]:
# ValueError: 'x0' must only have one dimension.